In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import os.path  # To manage paths
import sys
import time 
from datetime import datetime


import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.strategies as btstrats

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../.."))


import pandas as pd
import numpy as np

# Read the CSV file
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(os.getcwd())
print(df_raw.tail())
import joblib

scaler = joblib.load('./scaler.pkl')



from scripts.process import process

X, Y = process(df_raw, scaler, dev=False)
print(X.shape, Y.shape)

/home/elm19/Desktop/GoldSpot-Predictor/notebooks/new_approach_exp1
              close     open     high      low    volume
Date                                                    
2025-05-01  3222.20  3299.00  3300.60  3209.40  214690.0
2025-05-02  3243.30  3247.60  3277.00  3229.50  210990.0
2025-05-05  3322.30  3247.10  3346.70  3243.10  184900.0
2025-05-06  3409.06  3345.70  3444.50  3332.10   36710.0
2025-05-07  3399.99  3409.27  3409.94  3368.05  157510.0


2025-06-04 21:27:01.141201: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 21:27:01.150972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 21:27:01.172001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749068821.214658   29447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749068821.227893   29447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749068821.262339   29447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

(3855, 20, 12) (3855, 3)


In [ ]:
#!/usr/bin/env python3

import numpy as np
from keras.models import load_model

# 1) Load your saved model
model = load_model('./regulized_model.keras')

# 2) Prepare your input data as a NumPy array of shape (n_samples, seq_len, n_features)
#    For example:
# X_new = np.load('X_new.npy')

# 3) Get class‐probabilities and class indices
proba   = model.predict(X)             
pred_ix = np.argmax(proba, axis=1)         

# 4) (Optional) Map back to labels
class_map = {0: 'sell', 1: 'hold', 2: 'buy'}
pred_labels = [class_map[i] for i in pred_ix]


121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
float32


In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.evaluate(X, Y, verbose=0)

[0.5414470434188843, 0.7553825974464417]

In [32]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = dict(
        atr_period=14,
        atr_multiplier=1.5,  # for stop-loss
        risk_per_trade = 0.01
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.atr = bt.ind.ATR(self.data, period=self.p.atr_period)
        self.order = None
        
        


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status == order.Canceled:
            print("Order was canceled.")

        elif order.status == order.Margin:
            print("Order was rejected due to insufficient margin.")

        elif order.status == order.Rejected:
            print("Order was rejected by the broker.")

        # Write down: no pending order
        self.order = None

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        if not self.position and  len(self) < len(pred_labels) and len(self) >= 34:
            self.log(self.dataclose)
        
        # time.sleep(1)

    def place_order(self, order_type, atr_value):
        stop_loss_distance = atr_value * self.p.atr_multiplier

        capital = self.broker.getvalue()
        risk_amount = capital * self.p.risk_per_trade
        size = risk_amount / stop_loss_distance
        self.price = self.data.close[0]
        
        if order_type == "buy":
            self.stop_price = self.price - stop_loss_distance
            # self.buy(size=size,  trailamount=stop_loss_distance)

            self.buy_bracket(
                price=self.price,
                size=size,
                # exectype=bt.Order.TrailingStop,
                # trailamount=stop_loss_distance,
                stopprice=self.stop_price,
                limitprice=self.price + stop_loss_distance,
            )
        elif order_type=="sell":
            self.stop_price = self.price + stop_loss_distance
            # self.sell(size=size,  trailamount=stop_loss_distance)
            self.sell_bracket(
                price=self.price,
                size=size,
                limitprice=self.price - stop_loss_distance,
                stopprice=self.stop_price
            )
        

In [33]:
# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(TestStrategy)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
# cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = '../../data/processed-data/g_2010_2023.csv'

# Create a Data Feed# Add a strategy
data = bt.feeds.GenericCSVData(
    dataname=datapath,
    dtformat=('%Y-%m-%d'),
    datetime=0,
    time=-1,
    openinterest=-1,
    high=3,
    low=4,
    close=1,
    volume=5,
    open=2,
    # nullvalue=0.0
)
# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
strats = cerebro.run()
# cerebro.plot()
# strat = strats[0]


# print("Total Trades:", strat.analyzers.trades.get_analysis()['total']['total'])
# print("Net Profit:", strat.analyzers.trades.get_analysis()['pnl']['net']['total'])
# print("Sharpe Ratio:", strat.analyzers.sharpe.get_analysis())
# print("Max Drawdown %:", strat.analyzers.drawdown.get_analysis()['max']['drawdown'])
# print("Max Drawdown Value:", strat.analyzers.drawdown.get_analysis()['max']['moneydown'])
# print("SQN:", strat.analyzers.sqn.get_analysis())
# print("Returns:", strat.analyzers.returns.get_analysis()['rtot'])

# pyfoliozer = strat.analyzers.getbyname('pyfolio')
# returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()


# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00


2010-02-22, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-02-23, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-02-24, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-02-25, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-02-26, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-01, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-02, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-03, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-04, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-05, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-08, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-09, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-10, <backtrader.linebuffer.LineBuffer object at 0x7932a98dc410>
2010-03-11, <backtrader.linebuffer.LineBuffer object at 0x7932a9